<a href="https://colab.research.google.com/github/Zarif123/SSLM-Project/blob/main/chess_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
%%capture
!pip install transformers
!pip install python-chess

In [11]:
from google.colab import drive
drive.mount('/content/gdrive')
folder = "/content/gdrive/MyDrive/Statistical_LM_Group_Folder"

csv_file = "chess_data.csv"
csv_path = f"{folder}/{csv_file}"

model_file = "model.pth"
model_path = f"{folder}/{model_file}"

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [12]:
from transformers import BertTokenizer, AutoTokenizer, BertModel, BertConfig, get_linear_schedule_with_warmup
import torch.optim as optim
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
from tqdm import tqdm

In [13]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [14]:
params = {
    "batch_size": 2,
    "epochs": 5,
    "learning_rate": 0.0005,
    "warmup_steps": 0.01,
    "epsilon": 1e-8,
    "accum_iter": 4
}

class Params:
    def __init__(self, **kwargs):
        for key, value in kwargs.items():
            setattr(self, key, value)

params = Params(**params)

In [15]:
class BertClassifier(nn.Module):
    def __init__(self, dropout=0.5, num_classes=11):
        super(BertClassifier, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-cased')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, num_classes)
        self.relu = nn.ReLU()

    def forward(self, input_id, mask):
        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)

        return final_layer

In [16]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, df, labels, tokenizer):
        self.labels = [labels[label] for label in df['category']]
        self.texts = [tokenizer(text, 
                               padding='max_length', max_length = 512, truncation=True,
                                return_tensors="pt") for text in df['text']]
                                
    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y

In [17]:
def train(model, train_data, val_data, learning_rate):
    train, val = Dataset(train_data), Dataset(val_data)

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=params.batch_size, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=params.batch_size)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr = params.learning_rate)
    num_batches = len(train_dataloader)

    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps = params.warmup_steps, num_training_steps = num_batches * params.epochs)

    for epoch_num in range(params.epochs):
        total_acc_train = 0
        total_loss_train = 0

        for batch_idx, train_input, train_label in tqdm(train_dataloader.itertuples(), total=num_batches):

            train_label = train_label.to(device)
            mask = train_input['attention_mask'].to(device)
            input_id = train_input['input_ids'].squeeze(1).to(device)

            output = model(input_id, mask)
            
            batch_loss = criterion(output, train_label.long())
            total_loss_train += batch_loss.item()
            
            acc = (output.argmax(dim=1) == train_label).sum().item()
            total_acc_train += acc

            batch_loss.backward()

            # Gradient Accumulation
            if ((batch_idx + 1) % params.accum_iter == 0) or (batch_idx + 1 == num_batches):
              optimizer.step()
              scheduler.step()
              model.zero_grad()
        
        total_acc_val = 0
        total_loss_val = 0

        with torch.no_grad():

            for val_input, val_label in val_dataloader:

                val_label = val_label.to(device)
                mask = val_input['attention_mask'].to(device)
                input_id = val_input['input_ids'].squeeze(1).to(device)

                output = model(input_id, mask)

                batch_loss = criterion(output, val_label.long())
                total_loss_val += batch_loss.item()
                
                acc = (output.argmax(dim=1) == val_label).sum().item()
                total_acc_val += acc
        
        print(
            f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} \
            | Train Accuracy: {total_acc_train / len(train_data): .3f} \
            | Val Loss: {total_loss_val / len(val_data): .3f} \
            | Val Accuracy: {total_acc_val / len(val_data): .3f}')